## Example on how to evaluate a vision encoder with the Hummingbird or Dense NN Retrieval Evaluation

<a href="https://githubtocolab.com/vpariza/open-hummingbird-eval/blob/main/examples/hbird_eval_example_faiss_gpu.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### 1. Install required Libraries

In [ ]:
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121
!pip install lightning==2.4.0
!pip install torchmetrics==1.7.0
!pip install tqdm==4.67.1 # optional
!pip install scipy==1.15.2
!pip install joblib==1.4.2
!pip install numpy==1.26.4
!pip install triton==2.2.0
!pip install faiss-gpu-cu12
!pip install gdown

!pip uninstall -y thinc

**Important**: After the installation step please restart the Runtime/Kernel before continuing with the step 2.

### 2. Clone our repository

In [ ]:
!git clone https://github.com/vpariza/open-hummingbird-eval.git

In [ ]:
# Move to the repository folder
%cd open-hummingbird-eval

### 3. Install repo

In [ ]:
!pip install .

### 3. Download your preferred dataset

In [ ]:
# Used to delete previously downloaded and used
# !rm -rf ./VOCSegmentation
# !rm voc_data.zip

#### Option 1: Tiny Pascal VOC

In [ ]:
# Download the tiny Pascal VOC
!wget -O voc_data.zip "https://drive.google.com/uc?id=1o9evnop5BJnT47jwFk9cUp3TxHRdNpVN"

#### Option 2: Mini Pascal VOC

In [ ]:
# Download the mini Pascal VOC
!wget -O voc_data.zip "https://drive.google.com/uc?id=1GoRApuZzsM8a5nTa-wyEsLPOXPJuzbcT"

#### Option 3: Pascal VOC

In [ ]:
# Download the full Pascal VOC
!wget -O voc_data.zip "https://drive.google.com/uc?id=1LITii48SINBFAankzh_HlYWx5jCJzG4X"

### 4. Unzip Contents of zip Dataset

In [ ]:
!unzip voc_data.zip

### 5. Evaluate a preferred model on the downloaded dataset

In [ ]:
import torch
from hbird.hbird_eval import hbird_evaluation

In [ ]:
# Parameters for the model dino
device = 'cuda'
input_size = 224
batch_size = 64
patch_size = 16
embed_dim = 384
model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

In [ ]:
# Dataset Configurations
dataset_name='voc'
data_dir='./VOCSegmentation'
train_fs_path= './VOCSegmentation/sets/trainaug.txt'
val_fs_path= './VOCSegmentation/sets/val.txt'

In [ ]:
def extract_dino_features(model, imgs):
    return model.get_intermediate_layers(imgs)[0][:, 1:], None

In [ ]:
hbird_miou = hbird_evaluation(model.to(device),
        d_model=embed_dim,        # size of the embedding feature vectors of patches
        patch_size=patch_size,
        batch_size = batch_size,
        input_size=input_size,
        augmentation_epoch=1,     # how many iterations of augmentations to use on top of the training dataset in order to generate the memory
        device=device,
        return_knn_details=False, # whether to return additional NNs details
        nn_method='faiss',
        n_neighbours=30,         # the number of neighbors to fetch per image patch
        nn_params=None,           # Other parameters to be used for the k-NN operator
        ftr_extr_fn=extract_dino_features,           # function that extracts features from a vision encoder on images
        dataset_name=dataset_name,       # the name of the dataset to use, currently only Pascal VOC is included.
        data_dir=data_dir,    # path to the dataset to use for evaluation
        memory_size=None,
        train_fs_path=train_fs_path,
        val_fs_path=val_fs_path)
print('Hummingbird Evaluation (mIoU):', hbird_miou)